On va se servir d’un champ mutable pour définir un type `'a dynarray` de
tableaux dynamiques, c’est-à-dire de tableaux dont la taille pourra
changer en ajoutant des éléments dedans à la manière des `list` de
`Python`.

Pour cela, on va d’abord commencer à définir des fonctions sur les
`array` (qui existent pour beaucoup dans la bibliothèque standard mais
c’est surtout l’occasion de s’entrainer).

# Préliminaires - fonctions de manipulations de tableaux

**Question 1**

Écrire une fonction `array_blit` telle que
`array_blit tab1 debut1 tab2 debut2 nb` copie les `nb` éléments de
`tab1` à partir de l’indice `debut1` dans le tableau `tab2` à partir de
l’indice `debut2`.

On appelera toujours cette fonction en faisant en sorte qu’il n’y ait
pas d’accès érronés en mémoire.

In [10]:
let array_blit (tab1:'a array) (debut1:int) 
               (tab2:'a array) (debut2:int) (nb:int) : unit =
               for i=0 to nb-1 do
                        tab2.(debut2+i) <- tab1.(debut1+i)
                done


val array_blit : 'a array -> int -> 'a array -> int -> int -> unit = <fun>

In [12]:
let _ =
    let t1 = Array.init 5 (fun i -> i) in
    let t2 = Array.init 6 (fun i -> i+1) in
    array_blit t1 3 t2 1 2;
    assert(t2 = [|1;3;4;4;5;6|]);
    let t1 = [| 'a';'b';'c' |] in
    let t2 = [| 'd'; 'e' |] in
    array_blit t1 1 t2 0 2;
    assert(t2 = [|'b';'c'|]);
    "Valide"

- : string = "Valide"

**Question 2**

Écrire une fonction `array_extend` telle que l’appel
`array_extend tab debut nb` renvoie un **nouveau** tableau contenant le
même contenu que `tab` sauf que l’on a rajouté `nb` éléments à partir de
l’indice `debut`. La valeur des élements rajoutés n’est pas signifiante.

On suppose que `tab` est non vide.

Si `debut` vaut la longueur de `tab`, les nouveaux élèments sont
rajoutés après la recopie de `tab`.

On utilisera `array_blit`.

In [37]:
let array_extend (tab:'a array) (debut:int) (nb:int) : 'a array =
    let len = Array.length tab in
    let ret = Array.make (nb+len) tab.(0) in
    begin
        array_blit tab 0 ret 0 debut;
        array_blit tab (debut) ret (debut+nb) (len - debut) ; 
        ret
    end

val array_extend : 'a array -> int -> int -> 'a array = <fun>

In [38]:
let _ =
    let t = Array.init 5 (fun i -> i) in
    let t2 = array_extend t 3 5 in
    assert(Array.length t2 = 10 
        && Array.sub t2 0 3 = [|0;1;2|] 
        && Array.sub t2 8 2 = [|3;4|]);
    let t = Array.init 5 (fun i -> char_of_int (i+97) ) in
    let t2 = array_extend t 0 3 in
    assert(Array.length t2 = 8 && Array.sub t2 3 5 = t);
    let t2 = array_extend t 5 3 in
    assert(Array.length t2 = 8 && Array.sub t2 0 5 = t);
    "Valide"


- : string = "Valide"

En exécutant le code suivant, vous devriez obtenir
`[|0; 1; 2; 0; 0; 0; 0; 0; 3; 4|]`.

In [39]:
let t = Array.init 5 (fun i -> i)
in array_extend t 3 5

- : int array = [|0; 1; 2; 0; 0; 0; 0; 0; 3; 4|]

**Question 3**

Écrire une fonction `array_concat` telle que l’appel
`array_concat tab1 tab2` renvoie le **nouveau** tableau obtenu en
plaçant les éléments de `tab2` à la suite de ceux de `tab1`.

In [46]:
let array_concat (tab1:'a array) (tab2:'a array) : 'a array =
    if tab1 = [||] then tab2 
    else if tab2 = [||] then tab1 
    else 
    let l1 = Array.length tab1 in
    let l2 = Array.length tab2 in
    let ret = Array.make (l1+l2) tab1.(0) in
    array_blit tab1 0 ret 0 l1;
    array_blit tab2 0 ret l1 l2;
    ret

val array_concat : 'a array -> 'a array -> 'a array = <fun>

In [47]:
let _ =
    assert(array_concat [|1;2|] [|3|] = [|1;2;3|]);
    assert(array_concat [||] [||] = [||]);
    assert(array_concat [||] [|1|] = [|1|]);
    assert(array_concat [|1|] [||] = [|1|]);
    "Valide"


- : string = "Valide"

**Question 4**

Écrire une fonction `array_delete` telle que l’appel
`array_delete tab debut nb` renvoie un **nouveau** tableau obtenu en
supprimant `nb` élèments de `tab` à partir de l’indice `debut`.

On suppose que `nb` \> 0 et donc que `tab` est non vide.

In [73]:
let array_delete (tab : 'a array) (debut : int) (nb : int) : 'a array =
                let lt = Array.length tab in
                let ret = Array.make (lt - nb ) tab.(0) in
                array_blit tab 0 ret 0 debut;
                array_blit tab (debut+nb) ret debut (lt-debut-nb) ;
                ret

val array_delete : 'a array -> int -> int -> 'a array = <fun>

In [76]:
let _ =
    assert(array_delete [|1;2;3;4;5;6|] 2 2 = [|1;2;5;6|]);
    assert(array_delete [|1;2;3;4;5;6|] 0 3 = [|4;5;6|]);
    "Valide"


- : string = "Valide"

## Premier type de tableaux dynamiques

On définit le type suivant pour contenir un tableau dynamique. On va
devoir reallouer les élèments, donc on aura besoin d’accèder en écriture
au tableau qui les contient, le plus simple dans un premier temps est de
faire une référence sur un tableau.

*Note* le type est un type synonyme, par défaut, `OCaml` va afficher
`'a array ref` dans ses résultats mais on peut forcer l’affichage de
`'a dynarray` en spécifiant les types dans les fonctions comme on va le
faire en dessous.

In [77]:
type 'a dynarray = 'a array ref

type 'a dynarray = 'a array ref

Comme on va définir des tableaux en partant du tableau vide, comme en
Python avec `[]`, on peut se permettre de ne pas donner le type d’un des
élèments. En fait, contrairement à ce qui semble la fonction suivante
n’a pas un type magique, le `'a` sera instancié le plus vite possible,
ce qu’on peut voir avec le `'_weak1` dans le type.

**Attention** : vraiment comprendre ces notions de paramètres génériques
faibles est trop complexe et dépasse le cadre du cours de CPGE.

In [78]:
let dynarray_init () = ref [||]

val dynarray_init : unit -> 'a array ref = <fun>

In [79]:
let v = dynarray_init ()

val v : '_weak1 array ref = {contents = [||]}

In [80]:
v := [|1; 2|]

- : unit = ()

In [81]:
(* Maintenant v est un int dynarray *)
v

- : int array ref = {contents = [|1; 2|]}

On va maintenant définir des fonctions d’ajout et de suppresson
d’élèments dans le tableau. Pour cela, on va reallouer le tableau grâce
aux fonctions de la partie précédente. **Attention** rappel dans
`array_extend` on a supposé le tableau non vide, mais ici il faudra
surement faire un cas à part ici.

**Question 5**

Écrire une fonction `dynarray_append` telle que l’appel
`dynarray_append tab x` ajoute `x` à la fin du tableau `tab`.

In [83]:
let dynarray_append (tab : 'a dynarray) (x : 'a) : unit =
    tab.contents <- array_concat tab.contents [|x|]

val dynarray_append : 'a dynarray -> 'a -> unit = <fun>

In [84]:
let _ =
    let t = dynarray_init () in
    dynarray_append t 1;
    dynarray_append t 2;
    dynarray_append t 3;
    dynarray_append t 4;
    assert(!t = [|1;2;3;4|]);
    "Valide"


- : string = "Valide"

**Question 6**

Écrire une fonction `dynarray_pop` telle que l’appel à
`dynarray_pop tab` supprime le dernier élèment de `tab` et le renvoie.

In [121]:
let dynarray_pop (tab : 'a dynarray) : 'a =
    let lt = Array.length !tab in
    let c = tab.contents.(lt-1) in
    tab.contents <- array_delete !tab (lt - 1) 1; 
    c 

val dynarray_pop : 'a dynarray -> 'a = <fun>

In [122]:
let _ =
    let t = ref [|1;2;3;4|] in
    let x = dynarray_pop t in
    assert(x = 4 && !t = [|1;2;3|]);
    "Valide"

- : string = "Valide"

## Type efficace pour les tableaux dynamiques

Afin de résoudre les problèmes observés précedemment de complexité, on
va définir un nouveau type pour les tableaux en permettant d’avoir un
tableau contenant plus d’éléments que ce qui est vraiment utilisé.
Ainsi, si on a un tableau de taille 10 mais qui ne contient que 9
élèments réels, on pourra en temps constant rajouter un élèment. Pour
cela, on va avoir besoin non seulemetn de gérer les élèments du tableau,
mais aussi un paramètre `length` donnant la longueur logique,
c’est-à-dire le nombre d’éléments signifiants du tableau.

In [123]:
type 'a dynarray = {
    mutable elements : 'a array;
    mutable length : int
}

type 'a dynarray = { mutable elements : 'a array; mutable length : int; }

In [124]:
let dynarray_init () = { elements = [||]; length = 0 }

val dynarray_init : unit -> 'a dynarray = <fun>

**Question 7**

Écrire une fonction `dynarray_of_array` telle que l’appel
`dynarray_of_array tab` où `tab` est un `array` renvoie le dynarray le
contenant : on initialisera sa longueur à la longueur de `tab`.

In [125]:
let dynarray_of_array (tab : 'a array) : 'a dynarray = { elements = tab; length = Array.length tab}

val dynarray_of_array : 'a array -> 'a dynarray = <fun>

In [126]:
let _ =
    assert(dynarray_of_array [|1;2|] = { elements = [|1;2|]; length = 2 });
    "Valide"


- : string = "Valide"

**Question 8**

Écrire des fonctions `get` et `set` telles que l’appel `get tab i`
renvoie le ième élément du tableau contenu par `tab` et l’appel
`set tab i x` place la valeur `x` dans le tableau `tab` à l’indice `i`.

In [127]:
let get (tab : 'a dynarray) (i : int) : 'a = tab.elements.(i)

let set (tab : 'a dynarray) (i : int) (x : 'a) : unit = tab.elements.(i) <- x

val get : 'a dynarray -> int -> 'a = <fun>
val set : 'a dynarray -> int -> 'a -> unit = <fun>

In [128]:
let _ =
    let d = dynarray_of_array [|1;2|] in
    assert(get d 1 = 2);
    set d 1 3;
    assert(d.elements = [|1;3|]);
    "Valide"


- : string = "Valide"

Pour ajouter des élèments au tableau, on va regarder s’il y a de la
place `t.length < Array.length t.elements` et sinon on va **doubler** sa
taille. Si le tableau contenait 3 élèments, on va en allouer 6 donc 3 de
plus, ce qui va permettre de faire 2 ajouts derrière.

**Attention** : quand le tableau est vide, on ne pourra pas doubler sa
longueur, on se contente de faire un tableau de taille 1 dans ce cas.

**Question 9**

Écrire une fonction `dynarray_append` telle que l’appel
`dynarray_append tab x` ajoute `x` à la fin du tableau `tab` avec la
stratégie exposée.

In [145]:
let dynarray_append (tab : 'a dynarray) (x : 'a) : unit =
    tab.elements <- array_concat tab.elements tab.elements;
    tab.elements.(tab.length) <- x;
    tab.length <- tab.length + 1

val dynarray_append : 'a dynarray -> 'a -> unit = <fun>

In [146]:
let _ =
    let t = dynarray_init () in
    dynarray_append t 1;
    dynarray_append t 2;
    dynarray_append t 3;
    dynarray_append t 4;
    dynarray_append t 5;
    assert(Array.sub t.elements 0 5 = [|1; 2; 3; 4; 5|] 
        && t.length = 5);
    "Valide"


Exception: (Invalid_argument "index out of bounds")

**Question 10**

Écrire une fonction `dynarray_pop` telle que l’appel `dynarray_pop tab`
supprime le dernier élèment de `tab` et le renvoie.

On ne supprime rien physiquement, on se contente de changer la valeur
`length`.

In [141]:
let dynarray_pop (tab : 'a dynarray) : 'a =
        let c = tab.elements.(tab.length-1) in
        tab.length <- tab.length - 1;
        c 

val dynarray_pop : 'a dynarray -> 'a = <fun>

In [142]:
let _ =
    let t = { elements = [|1;2;3;4;5|]; length = 4 } in
    let x = dynarray_pop t in
    
    assert(x = 4 && t.elements = [|1;2;3;4;5|] && t.length = 3);
    "Valide"


- : string = "Valide"

**Question 11**

Prouver que si on effectue une suite de $n$ `append` sur un tableau vide
on fait $O(n)$ opérations élementaires.

Même question avec des `append` et des `pop`.

Si on effectue 1 milliard d’`append` suivi d’1 milliard de `pop`, la
taille physique sera toujours d’1 milliard d’éléments. Cela semble peu
efficace et on aimerait que la taille physique soit toujours proche du
nombre réél d’éléments.

**Question 12**

Comment peut-on modifier `dynarray_pop` pour s’assurer que le nombre
d’éléments réels est toujours proportionnel à la taille physique tout en
conservant un temps constant en moyenne ?

Faites la modification et faites une preuve assurant que cela
fonctionne.

In [ ]:
let lt = Array.length !tab in
    let c = tab.contents.(lt-1) in
    tab.contents <- array_delete !tab (lt - 1) 1; 
    c 